In [32]:
import os
import glob
import pandas as pd
import csv
import requests
import time
import random
from bs4 import BeautifulSoup

In [33]:
seed_path = os.path.join('data', 'seed')
top_songs_filename = os.path.join('data', 'raw', 'top_songs_with_lyrics.csv')

# Read data

In [34]:
all_files = glob.glob(seed_path + "/*.csv")
song_data = pd.concat(pd.read_csv(f) for f in all_files)
song_data.head()

,rank,artists,title
0,1,Maluma,Hawai
1,2,"Sech,Daddy Yankee,J Balvin Featuring ROSALIA,F...",Relacion
2,3,"J Balvin,Dua Lipa,Bad Bunny & Tainy",Un Dia (One Day)
3,4,Ozuna,Caramelo
4,5,Nio Garcia x Anuel AA x Myke Towers x Brray x ...,La Jeepeta


In [35]:
# Normalize artist and title and drop duplicate rows
song_data['artists'] = song_data['artists'].str.upper() 
song_data['title'] = song_data['title'].str.upper() 
song_data.drop_duplicates(subset=['title', 'artists'], keep='first', inplace=True)

song_data.head()

,rank,artists,title
0,1,MALUMA,HAWAI
1,2,"SECH,DADDY YANKEE,J BALVIN FEATURING ROSALIA,F...",RELACION
2,3,"J BALVIN,DUA LIPA,BAD BUNNY & TAINY",UN DIA (ONE DAY)
3,4,OZUNA,CARAMELO
4,5,NIO GARCIA X ANUEL AA X MYKE TOWERS X BRRAY X ...,LA JEEPETA


# Get lyrics from MusicMatch api

In [21]:
# TODO: Ignorar las canciones que no tienen lyrics
# Referencia de como usar el api de music match https://youtu.be/WFRzKmpepj4
base_url= 'https://api.musixmatch.com/ws/1.1/'
api_key = "3950aca40989ee9f7d05723278e3fdfd"

In [22]:
def get_lyric(row):
    title = row['title']
    artist = row['artists']
    lyric_matcher = 'matcher.lyrics.get?format=json&callback=callback'
    url = f"{base_url}/{lyric_matcher}&q_track={title}&q_artist={artist}&apikey={api_key}"
    # print(title, artist, url)
    
    response = requests.get(url)
    data = response.json() # lee la información en json (cuidado con jsonp, se tiene que hacer mayor procesamiento)
    if data['message']['header']['status_code'] == 200:
        lyric = data['message']['body']['lyrics']['lyrics_body'].replace('******* This Lyrics is NOT for Commercial use *******', '')
        # print(lyric)
        
        row['lyric'] = lyric.replace('(1409620732275)',' ')
        row['explicit'] = data['message']['body']['lyrics']['explicit']
        row['lyric_provider'] = 'MusicMatch'
    return row

def get_music_genre(row):
    title = row['title']
    artist = row['artists']
    url = f"{base_url}/matcher.track.get?format=json&callback=callback&q_track={title}&q_artist={artist}&apikey={api_key}"
    # print(title, artist, url)
    
    response = requests.get(url)
    data = response.json()
    # print(data)
    if data['message']['header']['status_code'] == 200:
        track = data['message']['body']['track']
        # print(track)
        
        row['musixmatch_rating'] = track['track_rating']
        # row['artist_name'] = track['artist_name']
        # row['track_name'] = track['track_name']

        if len(track['primary_genres']['music_genre_list']) > 0:
            row['genre'] = track['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name_extended']
    return row

In [23]:
song_data = song_data.apply(get_lyric, axis=1)
song_data = song_data.apply(get_music_genre, axis=1)

# Get lyrics from Google api

In [36]:
def get_google_lyrics(row):
    # Simulate chrome, this prevent that google does not sent g-expandable-content elements
    headers = {
        # 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
    # Generate google url
    title = row['title']
    artist = row['artists']
    page = requests.get(f"https://www.google.com/search?q={artist}-{title}", headers=headers)
    
    # if does not return response 200 exit
    if page.status_code != 200:
        print('Blocked: ', page.status_code)
        return row
    
    # Parse html
    html = BeautifulSoup(page.text, 'html.parser')
     
    # Find music match lyrics
    lyrics_select = html.select("div[data-lyricid*='Musixmatch'] > g-expandable-content:nth-child(2)")
    print(lyrics_select)
    if len(lyrics_select) > 0:  # If google use musicmatch
        lyrics_container = lyrics_select[0]
        #se agrega timer ramdom para que no se bloquee le IP
        time.sleep(random.uniform(0,3))
        print(title, artist, "GOOGLE")
        
        google_lyric = []
        for prose in lyrics_container.select("div"):
            for verse in prose.select("span"):
                google_lyric.append(verse.text + '\n')  # End of verse
            google_lyric.append('\n')  # End of prose
        google_lyric = ''.join(google_lyric) 

        row['lyric'] = google_lyric
        row['lyric_provider'] = 'Google'
    else:  # Todo: other Google lyrics provider
            pass  
    
    return row

In [37]:
song_data = song_data.apply(get_google_lyrics, axis=1)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Pocas amistades, a la más sincera la llevo en la lista</span><br/><span jsname="YS01Ge">Pocos los amigos, los que me ayudaron y eso no se olvida</span><br/><span jsname="YS01Ge">Los que se pasaron, ahora vienen solos, es hipocresía</span><br/><span jsname="YS01Ge">No quieren lo que tengo, quieren no lo tenga y eso es pura envidia</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Yo no tengo nada, pero mi palabra vale más que todo</span><br/><span jsname="YS01Ge">Para un buen amigo, siempre está mi mano y esa vale oro</span><br/><span jsname="YS01Ge">A los que me aprecian, saben bien de sobra que estamos pa' todo</span><br/><span jsname="YS01Ge">Y cuando se ocupe, si se ocupa Lupe, le damos con Toño</s

PALABRA DE HOMBRE EL FANTASMA GOOGLE
[]
[]
[]
[]
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Sólo tú, provocas un suspiro</span><br/><span jsname="YS01Ge">Y me haces verte en cada lado que yo miro</span><br/><span jsname="YS01Ge">Sólo tú, conviertes lo imposible en realidad</span><br/><span jsname="YS01Ge">Eres mi necesidad</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Sólo tú, me llenas los vacíos</span><br/><span jsname="YS01Ge">Y hace cuántas vidas yo te habría elegido</span><br/><span jsname="YS01Ge">¿Por qué, tú?, esa pregunta fácil es de responder</span><br/><span jsname="YS01Ge">Porque ya, te estaba amando, incluso antes de nacer</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Sólo tú me haces sentir, lo que realmente es amor</span><br/><span j

[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Me está doliendo</span><br/><span jsname="YS01Ge">Tu desprecio</span><br/><span jsname="YS01Ge">No tener ni una razón de ti</span><br/><span jsname="YS01Ge">Por ser un tonto</span><br/><span jsname="YS01Ge">Hoy te he perdido</span><br/><span jsname="YS01Ge">El frio de la soledad ya lo sentí</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Pero este orgullo</span><br/><span jsname="YS01Ge">No me permite buscarte las consecuencias</span><br/><span jsname="YS01Ge">Las tomo con intereses</span><br/><span jsname="YS01Ge">Y en la garganta siento un nudo que me ahoga</span><br/><span jsname="YS01Ge">Y las ganas de estar contigo</span><br/><span jsname="YS01Ge">Me traicionan</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01

ME HICISTE UN BORRACHO EDWIN LUNA Y LA TRAKALOSA DE MONTERREY GOOGLE
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Ya no te extraño</span><br/><span jsname="YS01Ge">Me dueles a ratos</span><br/><span jsname="YS01Ge">Te es diferente</span><br/><span jsname="YS01Ge">Y justamente</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Me dueles a veces</span><br/><span jsname="YS01Ge">Al recordar</span><br/><span jsname="YS01Ge">Que te di de más</span><br/><span jsname="YS01Ge">Y tú indiferente</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Te creí que estaba roto tu corazón</span><br/><span jsname="YS01Ge">Y no estaba roto</span><br/><span jsname="YS01Ge">Estaba vacío, y yo de tonto, terco</span><br/><span jsname="YS01Ge">Quería completarlo con el mío</span></div>

MENTIRAS REMMY VALENZUELA GOOGLE
[]
[]
[]
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">No te sientas mal</span><br/><span jsname="YS01Ge">No es tu culpa, la verdad</span><br/><span jsname="YS01Ge">Si él no te supo cuidar como lo hago yo</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Te quiso comprar</span><br/><span jsname="YS01Ge">Solo con lo material</span><br/><span jsname="YS01Ge">Y olvidó que lo importante es el amor</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Dile que ya no lo necesitas</span><br/><span jsname="YS01Ge">Y que todas esas prendas que un día te compró, yo soy quién te las quita</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Dile que ya se vaya olvidando</span><br/><span jsname="YS01Ge">Que lo que él nunca te

...

TENGO QUE COLGAR BANDA MS GOOGLE
[]
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Vamos a bailar algo que está perrón</span><br/><span jsname="YS01Ge">Que toda la gente brinca de emoción</span><br/><span jsname="YS01Ge">Se mueven muy rico con esta canción</span><br/><span jsname="YS01Ge">Tocamos el ritmo desde el corazón</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Del centro del norte y de Mazatlán</span><br/><span jsname="YS01Ge">También del sureste, hay gente invitada</span><br/><span jsname="YS01Ge">El pato se mueve con mucha emoción</span><br/><span jsname="YS01Ge">Bailando este paso el pasito perrón</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Vamos a bailar algo que está perrón</span><br/><span jsname="YS01Ge">Que toda la gente brinca de emoc

NO ME HUBIERA ENAMORADO CORNELIO VEGA Y SU DINASTIA GOOGLE
[]
[]
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Es el sabor del Ensamble</span><br/><span jsname="YS01Ge">Sí, señor</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Dicen tus jefes que a mí no me quieren</span><br/><span jsname="YS01Ge">Y yo me awito y me salgo a beber</span><br/><span jsname="YS01Ge">Dicen que yo soy un gran mujeriego</span><br/><span jsname="YS01Ge">Y que te busco sólo pa' joder</span></div><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Joden y joden, no sé lo que quieren</span><br/><span jsname="YS01Ge">Si yo te amo y siempre te amaré</span><br/><span jsname="YS01Ge">Dicen tus jefes que yo soy un vago</span><br/><span jsname="YS01Ge">Que soy borracho y un loco, también</span></div><div

[]
[<g-expandable-content aria-hidden="true" data-eb="0" data-mt="0" jsaction=";rcuQ6b:npT2md" jscontroller="wrFDyc" jsname="WbKHeb" jsshadow="" style="display:none"><span jsslot=""><div class="ujudUb" jsname="U8S5sf"><span jsname="YS01Ge">Siéntate, escuchame prepárate</span><br/><span jsname="YS01Ge">Lo que voy a decirte</span><br/><span jsname="YS01Ge">Lo he pensado muy bien</span><br/><span jsname="YS01Ge">Yo no la busqué solo la encontré y me enamoré</span><br/><span jsname="YS01Ge">Tu me hiciste a un lado y por eso fue que yo me fije</span><br/><span jsname="YS01Ge">En su sonrisa, su belleza ya no te quiero ni contar</span><br/><span jsname="YS01Ge">Como es cuando me besa</span><br/><span jsname="YS01Ge">Como es en la cama</span><br/><span jsname="YS01Ge">Ya no me preguntes qué es lo que ella tiene que no tienes tu</span><br/><span jsname="YS01Ge">Que como me mira</span><br/><span jsname="YS01Ge">Qué como se llama</span><br/><span jsname="YS01Ge">Y si me hace el amor mejor que tu<

[]
[]
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429


Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocked:  429
Blocke

In [40]:
song_data.tail(100)

,artists,lyric,lyric_provider,rank,title
1900,LOS TIGRES DEL NORTE,NaN,NaN,1901,EL EJEMPLO
1901,PATRULLA 81,NaN,NaN,1902,ERES DIVINA
1902,LOS ANGELES AZULES,NaN,NaN,1903,ENTREGA DE AMOR FT SAUL HERNANDEZ
1903,YANET GUADALUPE Y SU GRUPO,NaN,NaN,1904,EL ESTUPIDO
1904,LOS TEMERARIOS,NaN,NaN,1905,LA MEJOR QUE SOÑE
...,...,...,...,...,...
1995,CONJUNTO PRIMAVERA,NaN,NaN,1996,PERDONAME MI AMOR
1996,JUAN GABRIEL,NaN,NaN,1997,SE ME OLVIDO OTRA VEZ
1997,LOS CADETES DE LINARES,NaN,NaN,1998,LOS DOS AMIGOS
1998,TOÑO LIZARRAGA,NaN,NaN,1999,LOS GUSTOS QUE TIENE EL MUCHACHO


# Get LyricFind from Google api

In [ ]:
def get_google_lyrics(row):
    # Simulate chrome, this prevent that google does not sent g-expandable-content elements
    headers = {
        # 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
    # Generate google url
    title = row['title']
    artist = row['artists']
    page = requests.get(f"https://www.google.com/search?q={artist}-{title}", headers=headers)
    
    # if does not return response 200 exit
    if page.status_code != 200:
        print('Blocked: ', page.status_code)
        return row
    
    # Parse html
    html = BeautifulSoup(page.text, 'html.parser')
     
    # Find music match lyrics
    lyrics_select = html.select("[data-lyricid]")#
    print(lyrics_select)
    if len(lyrics_select) > 0:  # If google use LyricFind
        lyrics_container = lyrics_select[0]
        #se agrega timer ramdom para que no se bloquee le IP
        time.sleep(random.uniform(0,3))
        print(title, artist, "GOOGLE")
        
        google_lyric = []
        for prose in lyrics_container.select("div"):
            for verse in prose.select("span"):
                google_lyric.append(verse.text + '\n')  # End of verse
            google_lyric.append('\n')  # End of prose
        google_lyric = ''.join(google_lyric) 

        row['lyric'] = google_lyric
        row['lyric_provider'] = 'Google'
    else:  # Todo: other Google lyrics provider
            pass  
    
    return row

In [ ]:
song_data = song_data.apply(get_google_lyrics, axis=1)

# Export lyrics data

In [ ]:
song_data.to_csv(
    top_songs_filename, 
    index=False, 
    quoting=csv.QUOTE_NONNUMERIC  # Prevent errors on strings with , and \n characters
)